<a href="https://colab.research.google.com/github/justxperimenting/Pytorch_tutorial/blob/main/vid7_NN_training_on_GPU_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# set random seed for reproductibility
torch.manual_seed(42)

In [ ]:
# Steps
"""
1. Check GPU Availability

2. Move the model to GPU

3. Modify the Training loop by Moving Data to GPU

4.

"""

In [4]:
# check for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device : {device}')

Device : cuda


In [9]:
df = pd.read_csv('/content/fashion-mnist_train.csv')

In [10]:
df.shape

(60000, 785)

In [15]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [16]:
X.shape

(60000, 784)

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [18]:
X_train.shape

(48000, 784)

In [19]:
X_train = X_train/255.0
X_test = X_test/255.0

In [33]:
class CustomDataset(Dataset):
  def __init__(self,features, labels):
   self.features = torch.tensor(features, dtype = torch.float32)
   self.labels = torch.tensor(labels, dtype = torch.long)

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,index):
    return self.features[index], self.labels[index]



In [34]:
X_train.shape

(48000, 784)

In [35]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [36]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)

In [37]:
len(train_loader)

1500

In [38]:
class MyNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(num_features,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
        # here softmax should be present but,
        # in pytorch crossentropyloss , softmax is already applied at end
    )

  def forward(self,x):
    return self.model(x)


In [39]:
learning_rate = 0.1
epochs = 100

In [41]:
# instatiate the model
model = MyNN(X_train.shape[1])

# move model to gpu
model = model.to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr = learning_rate)

In [42]:
# training loop
for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:
    # moving data to gpu
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # backward pass
    optimizer.zero_grad()
    loss.backward()

    # update the weights
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch : {epoch+1}, Loss : {avg_loss}')


Epoch : 1, Loss : 0.6442541582981746
Epoch : 2, Loss : 0.432057913094759
Epoch : 3, Loss : 0.3867000223423044
Epoch : 4, Loss : 0.3572725569754839
Epoch : 5, Loss : 0.3385145670423905
Epoch : 6, Loss : 0.3232740025669336
Epoch : 7, Loss : 0.30780292422821126
Epoch : 8, Loss : 0.29756631531069677
Epoch : 9, Loss : 0.28432619310791296
Epoch : 10, Loss : 0.274511347360909
Epoch : 11, Loss : 0.26553403203065196
Epoch : 12, Loss : 0.2601559265678128
Epoch : 13, Loss : 0.2529807674959302
Epoch : 14, Loss : 0.24344130228521924
Epoch : 15, Loss : 0.23930770195275544
Epoch : 16, Loss : 0.2330618276372552
Epoch : 17, Loss : 0.22821492086847622
Epoch : 18, Loss : 0.22220238779857754
Epoch : 19, Loss : 0.217659245159477
Epoch : 20, Loss : 0.21119479821901768
Epoch : 21, Loss : 0.20388950807166598
Epoch : 22, Loss : 0.2011286676240464
Epoch : 23, Loss : 0.19966202997385213
Epoch : 24, Loss : 0.19421586274666092
Epoch : 25, Loss : 0.1880985551327467
Epoch : 26, Loss : 0.18550730106048285
Epoch : 27,

In [43]:
# evaluate the model
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [46]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features,batch_labels in test_loader:
    # moving the data to gpu
    batch_features,batch_labels = batch_features.to(device), batch_labels.to(device)

    output = model(batch_features)

    _,predicted = torch.max(output,1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(f'Accuracy : {correct/total}')



Accuracy : 0.8824166666666666
